### 데이터 값과 기업정보 최종 결과
* 결산년도 2017년 ~ 2021년
* 상장법인(유가증권+코스닥)

* 데이터 값 목록 가져오기

In [1]:
import pandas as pd

# 데이터값 파일
file_path = "D:/PythonProject/data-gatherer/dart_fs_notes/items_from_section/items_all.csv"
df_items_all = pd.read_csv(file_path, dtype=object)

# 기업개황정보 파일 읽기
file_path = "D:/PythonProject/data-gatherer/dart_fs_notes/company/corp_info_all.csv"
df_corp_info_all = pd.read_csv(file_path, dtype=object)

In [2]:
print(df_items_all.shape[0])
df_items_all.head(3)

2444


,corp_code,stock_code,corp_cls,corp_name,rpt_num,rpt_name,doc_title,doc_result,section_title,unit,val_a,val_b,item_result,doc_url
0,00100939,000860,Y,강남제비스코,20181231000114,[기재정정]사업보고서 (2013.12),NaN,No Financial Statement Notes,NaN,NaN,NaN,NaN,NaN,NaN
1,00100939,000860,Y,강남제비스코,20181231000137,[기재정정]사업보고서 (2017.12),5. 재무제표 주석,OK,21. 확정급여제도 :,백만원,23399.0,25631.0,"기금이 적립된 확정급여채무의 현재가치, 사외적립자산의 공정가치(*)",http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
2,00100939,000860,Y,강남제비스코,20181231000103,[기재정정]사업보고서 (2011.12),NaN,No Financial Statement Notes,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# df_items_all[df_items_all['corp_name'].str.contains('송원산업')]

* 데이터값과 기업개황정보 결합

In [3]:
df_result = pd.merge(df_items_all, df_corp_info_all[['corp_code', 'jurir_no', 'bizr_no', 'induty_code']], 
                     how='left', left_on='corp_code', right_on='corp_code')

In [4]:
# 금액을 백만원 단위로 변환
def change_val(v, unit):
    if unit == '백만원': amt = float(v)
    elif unit == '천원': amt = round(float(v) / 1000, 0)
    elif unit == '원': amt = round(float(v) / 1000000, 0)
    else: amt = 0
    return amt

In [5]:
# 금액을 백만원 단위로 변환한 항목을 추가
df_result['m_val_a'] = df_result.apply(lambda x : change_val(x['val_a'], x['unit']), axis=1)
df_result['m_val_b'] = df_result.apply(lambda x : change_val(x['val_b'], x['unit']), axis=1)

In [6]:
df_result.head(2)

,corp_code,stock_code,corp_cls,corp_name,rpt_num,rpt_name,doc_title,doc_result,section_title,unit,val_a,val_b,item_result,doc_url,jurir_no,bizr_no,induty_code,m_val_a,m_val_b
0,00100939,000860,Y,강남제비스코,20181231000114,[기재정정]사업보고서 (2013.12),NaN,No Financial Statement Notes,NaN,NaN,NaN,NaN,NaN,NaN,1801110003391,6058103330,20411,0.0,0.0
1,00100939,000860,Y,강남제비스코,20181231000137,[기재정정]사업보고서 (2017.12),5. 재무제표 주석,OK,21. 확정급여제도 :,백만원,23399.0,25631.0,"기금이 적립된 확정급여채무의 현재가치, 사외적립자산의 공정가치(*)",http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,1801110003391,6058103330,20411,23399.0,25631.0


In [20]:
# 컬럼순서 재배치
df_result = df_result[['induty_code', 'jurir_no', 'bizr_no', 'corp_code', 'stock_code', \
                       'corp_cls', 'corp_name', 'rpt_num', 'rpt_name', 'doc_title', 'doc_result', \
                       'section_title', 'unit', 'val_a', 'val_b', 'm_val_a', 'm_val_b', \
                       'item_result', 'doc_url']]
# 정렬
df_result.sort_values(by=['induty_code', 'jurir_no', 'corp_code', 'rpt_num'], inplace=True)
df_result.head(5)

,induty_code,jurir_no,bizr_no,corp_code,stock_code,corp_cls,corp_name,rpt_num,rpt_name,doc_title,doc_result,section_title,unit,val_a,val_b,m_val_a,m_val_b,item_result,doc_url
1109,00102,1101110104820,1048127881,00141875,006090,Y,사조오양,20170331003000,사업보고서 (2016.12),5. 재무제표 주석,OK,18. 종업원급여,천원,10802905.0,6213761.0,10803.0,6214.0,"확정급여채무의 현재가치, 사외적립자산의 공정가치",http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
1111,00102,1101110104820,1048127881,00141875,006090,Y,사조오양,20180402003435,사업보고서 (2017.12),5. 재무제표 주석,OK,18. 종업원급여,천원,12241819.0,7931515.0,12242.0,7932.0,"확정급여채무의 현재가치, 사외적립자산의 공정가치",http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
1110,00102,1101110104820,1048127881,00141875,006090,Y,사조오양,20190401003846,사업보고서 (2018.12),5. 재무제표 주석,OK,18. 종업원급여,천원,13764813.0,8456699.0,13765.0,8457.0,"확정급여채무의 현재가치, 사외적립자산의 공정가치",http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
1112,00102,1101110104820,1048127881,00141875,006090,Y,사조오양,20200330002828,사업보고서 (2019.12),5. 재무제표 주석,OK,18. 종업원급여,천원,16436592.0,9870436.0,16437.0,9870.0,"확정급여채무의 현재가치, 사외적립자산의 공정가치",http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
1108,00102,1101110104820,1048127881,00141875,006090,Y,사조오양,20210318001029,사업보고서 (2020.12),5. 재무제표 주석,OK,18. 종업원급여,천원,17390099.0,10501739.0,17390.0,10502.0,"확정급여채무의 현재가치, 사외적립자산의 공정가치",http://dart.fss.or.kr/report/viewer.do?rcpNo=2...


In [22]:
df_result_mfg = df_result[df_result['induty_code'].str[0:2].between('10', '34', inclusive='both')]
df_result_mfg.shape[0]

1949

In [25]:
corp_codes = df_result['corp_code'].unique()
len(corp_codes)

342

In [23]:
import sys
module_path = "D:\PythonProject\data-gatherer\dart_fs_notes"
sys.path.append(module_path)
import myutil

file_path = "D:/PythonProject/data-gatherer/dart_fs_notes/output/"
file_name = 'result_mfg_' + myutil.now_dt_str() + '.csv'
df_result_mfg.to_csv(file_path + file_name, index=False)